In [5]:
import os

# Dataset for the OAI images
data_folder1 = '/playpen-raid/data/OAI/0.C.2'
data_folder2 = '/playpen-raid/data/OAI/0.E.1'

data1 = os.listdir(data_folder1)
data2 = os.listdir(data_folder2)

print(len(data1), len(data2))
print(len(set.union(set(data1), set(data2))))

# dataset for the OAI-ZIB images
data_folder_zib = ''


2686 2110
4796


In [1]:
import pandas as pd

# dataset split for OAI dataset
pth_file = "/playpen-raid/data/OAI/contents.0E1.csv"
pth_df = pd.read_csv(pth_file)

SAG_3D_DESS_RIGHT = "SAG_3D_DESS_RIGHT"
SAG_3D_DESS_LEFT = "SAG_3D_DESS_LEFT"
sag_3d_dess_right_cases = pth_df[pth_df["SeriesDescription"] == SAG_3D_DESS_RIGHT]
sag_3d_dess_left_cases = pth_df[pth_df["SeriesDescription"] == SAG_3D_DESS_LEFT]

# get all the PIDs
PIDs = set(pth_df['ParticipantID'])

# get PIDs of OAI-ZIB dataset
ZIB_PID_to_Folder = {}
with open('/playpen-raid/qinliu/data/OAI-ZIB/doc/oai_mri_paths.txt') as f:
    lines = f.readlines()
    for line in lines:
        PID, folder = line.split(' ')[:2]
        PID = int(PID[:-1])
        ZIB_PID_to_Folder[PID] = folder

# dataset splitting according to PID
import random
random.seed(0)

ZIB_PIDs = list(ZIB_PID_to_Folder.keys())
random.shuffle(ZIB_PIDs)

ZIB_PIDs_train = ZIB_PIDs[:407]
ZIB_PIDs_val = ZIB_PIDs[407:457]
ZIB_PIDs_test = ZIB_PIDs[457:507]
print(len(ZIB_PIDs_train), len(ZIB_PIDs_val), len(ZIB_PIDs_test))

rest_PIDs = list(set(PIDs) - set(ZIB_PIDs))
random.shuffle(rest_PIDs)
rest_PIDs_train = rest_PIDs[:3431]
rest_PIDs_val = rest_PIDs[3431:3860]
rest_PIDs_test = rest_PIDs[3860:]
print(len(rest_PIDs_train), len(rest_PIDs_val), len(rest_PIDs_test))

train_set = set(ZIB_PIDs_train + rest_PIDs_train)
val_set = set(ZIB_PIDs_val + rest_PIDs_val)
test_set = set(ZIB_PIDs_test + rest_PIDs_test)

pd_data = []
for _, row in sag_3d_dess_right_cases.iterrows():
    folder, PID, right_or_left = row['Folder'], row['ParticipantID'], 'RIGHT' if row['SeriesDescription'] == 'SAG_3D_DESS_RIGHT' else 'LEFT'
    split = '?'
    if PID in train_set:
        split = 'Train'
    elif PID in val_set:
        split = 'Val'
    else:
        split = 'Test'        
    pd_data.append([PID, right_or_left, split, folder])


for _, row in sag_3d_dess_left_cases.iterrows():
    folder, PID, right_or_left = row['Folder'], row['ParticipantID'], 'RIGHT' if row['SeriesDescription'] == 'SAG_3D_DESS_RIGHT' else 'LEFT'
    split = '?'
    if PID in train_set:
        split = 'Train'
    elif PID in val_set:
        split = 'Val'
    else:
        split = 'Test'        
    pd_data.append([PID, right_or_left, split, folder])


pd_data_frame = pd.DataFrame(sorted(pd_data))
out_csv_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
pd_data_frame.to_csv(out_csv_file,  header=['pid', 'right_or_left', 'split', 'folder'], index=False)

407 50 50
3431 429 429


In [1]:
import sys
import SimpleITK as sitk
import numpy as np


def read_dicom_series(data_directory):
  """
  Read Dicom series from disk
  :param data_directory:  the folder containing all dicom series
  :return image: the 3D image volume
  """
  # Read the original series. First obtain the series file names using the image series reader.
  series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_directory)
  if not series_IDs:
    print("ERROR: given directory \"" + data_directory + "\" does not contain a DICOM series.")
    sys.exit(1)
  series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_directory, series_IDs[0])

  series_reader = sitk.ImageSeriesReader()
  series_reader.SetFileNames(series_file_names)

  # Configure the reader to load all of the DICOM tags (public + private):
  # By default tags are not loaded (saves time). If tags are loaded, the private tags are not loaded.
  # We explicitly configure the reader to load tags, including the private ones.
  series_reader.MetaDataDictionaryArrayUpdateOn()
  series_reader.LoadPrivateTagsOn()
  image = series_reader.Execute()

  return image


In [23]:
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import shutil


# read oai dataset 
oai_dataset_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
oai_df = pd.read_csv(oai_dataset_file)

# get all the test pids
oai_test_df = oai_df[oai_df['split'] == 'Val']
oai_test_pids_set = set(oai_test_df['pid'])

# get PIDs of OAI-ZIB dataset
zib_dataset_file = '/playpen-raid2/qinliu/data/OAI-ZIB/doc/oai_mri_paths.txt'
ZIB_PID_to_Folder = {}
with open(zib_dataset_file) as f:
    lines = f.readlines()
    for line in lines:
        PID, folder = line.split(' ')[:2]
        folder = folder.rstrip("\n")
        PID = int(PID[:-1])
        ZIB_PID_to_Folder[PID] = folder

zib_test_pids = set.intersection(set(ZIB_PID_to_Folder.keys()), oai_test_pids_set)
zib_test_pids = list(zib_test_pids)
zib_test_pids.sort()

# print(zib_test_pids)
# zib_test_pids = [9260515]

import os
from PIL import Image

image_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/val/image'
mask_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/val/annotations'
if os.path.isdir(image_folder):
    shutil.rmtree(image_folder)

if os.path.isdir(mask_folder):
    shutil.rmtree(mask_folder)

os.makedirs(image_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# generate images for the test set
for pid in zib_test_pids:
    print(pid)
    image_dicom_folder = os.path.join('/playpen-raid/data/OAI', ZIB_PID_to_Folder[pid])
    image = read_dicom_series(image_dicom_folder)
    image_npy = sitk.GetArrayFromImage(image)

    mask_mhd_path = os.path.join(
        '/playpen-raid2/qinliu/data/OAI-ZIB/segmentation_masks', 
        '{}.segmentation_masks.mhd'.format(pid)
    )
    mask = sitk.ReadImage(mask_mhd_path)
    mask_npy = sitk.GetArrayFromImage(mask)

    # print(image.GetDirection(), mask.GetDirection())
    # print(image_npy.shape, mask_npy.shape)

    mask_npy = np.transpose(mask_npy, (2,0,1))
    mask_npy = np.flip(mask_npy, axis=(0,1))

    mask_cartilage = np.zeros_like(mask_npy)
    mask_cartilage[mask_npy == 2] = 128
    mask_cartilage[mask_npy == 4] = 255

    for slice_idx in [40, 80, 120]:
        mask_slice = mask_cartilage[slice_idx]
        image_slice = image_npy[slice_idx]
        print(mask_slice.shape, image_slice.shape)

        if np.count_nonzero(mask_slice) > 20:
            # image_npy_rgb = np.stack([image_slice, image_slice, image_slice], axis=0)
            # mask_cartilage_rgb = np.stack([mask_slice, mask_slice, mask_slice], axis=0)

            # print(image_npy_rgb.shape)

            img = Image.fromarray(image_slice)
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(os.path.join(image_folder, '{}_{}.png'.format(pid, slice_idx)))

            msk = Image.fromarray(mask_slice)
            if msk.mode != 'RGB':
                msk = msk.convert('RGB')
            msk.save(os.path.join(mask_folder, '{}_{}.png'.format(pid, slice_idx)))

    # print(image_npy.shape, mask_npy.shape)

    # plt.subplot(131)
    # plt.imshow(image_npy[50], cmap='gray')

    # plt.subplot(132)
    # plt.imshow(mask_npy[50], cmap='gray')

    # plt.show()




9001104
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9006723
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9036287
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9036770
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9048192
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9053047
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9055836
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9071781
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9096402
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9140600
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9146462
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9157304
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9171766
(384, 384) (384, 384)
(384, 384) (384, 384)
(384, 384) (384, 384)
9173792
(384, 384) (384, 384)
(384, 38

In [6]:
dicom_folder = '/playpen-raid/data/OAI/0.C.2/9260515/20050425/10333513'
dicom_image = read_dicom_series(dicom_folder)
